---

Created for [Pricing and Hedging Derivative Securities: Theory and Methods](https://book.derivative-securities.org/)

Authored by
- Kerry Back, Rice University
- Hong Liu, Washington University in St. Louis
- Mark Loewenstein, University of Maryland
 
---

<a target="_blank" href="https://colab.research.google.com/github/math-finance-book/book-code/blob/main/23_PDEs.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<img src="https://www.dropbox.com/scl/fi/6hwvdff7ajaafmkpmnp0o/under_construction.jpg?rlkey=3dex2dx86anniqoutwyqashnu&dl=1" alt="Under Construction" width="400"/>


In [ ]:

import numpy as np

def crank_nicolson(a, y, L, z1, b1, zL, bL):
    u = np.zeros(L)
    b = np.zeros(L)
    c = np.zeros(L)
    z = np.zeros(L)

    u[0] = z1
    b[0] = b1
    for j in range(1, L - 1):
        z[j] = a[3] * y[j] + a[1] * y[j + 1] + a[2] * y[j - 1]
        u[j] = (a[2] * u[j - 1] + z[j]) / (a[0] - a[2] * b[j - 1])
        b[j] = a[1] / (a[0] - a[2] * b[j - 1])
    c[-1] = (zL + bL * u[-2]) / (1 - bL * b[-2])
    for j in range(L - 2, -1, -1):
        c[j] = u[j] + b[j] * c[j + 1]
    return c

def european_call_crank_nicolson(S0, K, r, sigma, q, T, N, M, Dist):
    dt = T / N
    dx = Dist / M
    dx2 = dx ** 2
    u = np.exp(dx)
    sig2 = sigma ** 2
    nu = r - q - sig2 / 2
    St = S0 * np.exp(Dist)
    Sb = S0 * np.exp(-Dist)
    a = np.zeros(4)
    a[0] = r / 2 + 1 / dt + sig2 / (2 * dx2)
    a[1] = sig2 / (4 * dx2) + nu / (4 * dx)
    a[2] = a[1] - nu / (2 * dx)
    a[3] = -a[0] + 2 / dt

    L = 2 * M + 1
    y = np.zeros(L)
    S = Sb
    y[0] = max(S - K, 0)
    for j in range(1, L):
        S *= u
        y[j] = max(S - K, 0)

    z1 = 0
    b1 = 1
    zL = St - St / u
    bL = 1
    CallV = crank_nicolson(a, y, L, z1, b1, zL, bL)

    for _ in range(N - 2, -1, -1):
        CallV = crank_nicolson(a, CallV, L, z1, b1, zL, bL)
    return CallV[M]

# Example usage
S0 = 100
K = 90
r = 0.05
sigma = 0.2
q = 0.02
T = 1
N = 100
M = 50
Dist = 3
Bar = 85

print("European Call Crank-Nicolson:", european_call_crank_nicolson(S0, K, r, sigma, q, T, N, M, Dist))

In [ ]:

def down_and_out_call_cn(S0, K, r, sigma, q, T, N, M, Dist, Bar):
    dx = Dist / M
    DistBot = np.log(S0) - np.log(Bar)
    NumBotSteps = int(np.ceil(DistBot / dx))
    dx = DistBot / NumBotSteps
    NumTopSteps = int(np.ceil(Dist / dx))
    DistTop = NumTopSteps * dx
    L = NumBotSteps + NumTopSteps + 1
    dt = T / N
    dx2 = dx ** 2
    u = np.exp(dx)
    sig2 = sigma ** 2
    nu = r - q - sig2 / 2
    St = S0 * np.exp(DistTop)
    a = np.zeros(4)
    a[0] = r / 2 + 1 / dt + sig2 / (2 * dx2)
    a[1] = sig2 / (4 * dx2) + nu / (4 * dx)
    a[2] = a[1] - nu / (2 * dx)
    a[3] = -a[0] + 2 / dt

    y = np.zeros(L)
    S = Bar
    y[0] = max(S - K, 0)
    for j in range(1, L):
        S *= u
        y[j] = max(S - K, 0)

    z1 = 0
    b1 = 0
    zL = St - St / u
    bL = 1
    CallV = crank_nicolson(a, y, L, z1, b1, zL, bL)

    for _ in range(N - 2, -1, -1):
        CallV = crank_nicolson(a, CallV, L, z1, b1, zL, bL)
    return CallV[NumBotSteps]

print("Down and Out Call CN:", down_and_out_call_cn(S0, K, r, sigma, q, T, N, M, Dist, Bar))